# Implementation in Python

## Introduction 

As we mentioned before, the main objective of this project is to implement an easy to use fractal renderer for educational purposes. The created tool lets user experiment with mentioned sets and study their behavior in an intuitive visual way. This visual representation of such sets stands out as a particularly intriguing aspect. Boundaries of Julia sets and related ones seem to take an amazingly natural and organic shapes, potentially prompting the reader to a deeper exploration of epistemological Platonism within the realm of mathematics. 

Unfortunately, when it comes to implementing most mathematical concepts numerically it's necessary to approximate the solution.
To visualize those sets will map their orbits to the color space. This will allow us to see the structure of the set. The mapping uses the number of iterations it took for the orbit to escape the circle of a given radius. The color of the pixel is then determined by the number of iterations it took the orbit to escape the given radius. In addition we will approximate the convergence of the orbit by calculating an arbitrary number of iterations, a value derived through meticulous testing of numerical effectiveness and optimization (in most cases set at 256, as it provides adequate results and allows for the utilization of the uint8 datatype in most cases).
Why is such approximation is valid? We can present it by simplifying the problem to a much more intuitive space R.

Iterating function f(x_k) = x_k-1^2, where x e R will showcase the behavior of orbits.

For values of x more than 1, the orbit will escape to infinity.
For x=1 the orbit will stay at 1.
For x between -1 and 1 the orbit will converge to 0.
For x less than -1 the orbit will also escape to infinity.

In this example we can call 0 and 1 fixed points of our iterative function. The point 0 is identified as an attractive fixed point, due to the fact that orbits starting in its vicinity converge towards it. Point 1 is a repulsive fixed point, owing to the fact that orbits around it diverge from it.

Values in the interval[-1,1] will not escape to infinity and thus constitute our set. In contrast values outside of this interval generate unbounded orbits, and are thus not included in the set.

This behavior seems intuitively justified by recognizing that 1 is an identity element for multiplication over R, and so it's natural for orbits to maintain their stationary state at 1. The operation of squearing (that is multiplication of a number by itself) ensures that values below 0 will be mirrored after the initial iteration to the positive side of the number line, so we can expect similar behavior to positive numbers, after the first iteration. Finally value of 0 being less than 1 (and to -1 in the mirrored domain), is anticipated to attract nearby orbits.

This example is very straightforward, so let's introduce complexity by modifying to our function. Now we can set our function to be f(x) = x^2 - 1. This small adjustment can provide us with examples of more interesting orbit behaviors.

For x=1 the orbit will transition from f(1) = 0 to f(0) = -1 and returning to f(-1) = 0. Such orbit is called periodic.
Orbits  initializing between -1 and 1 are incorporated in our set. Moreover orbits starting in proximity of this interval tend to fall into it during initial iterations, thus being included in the set as well.
Many orbits will exhibit seemingly chaotic behavior before stabilizing. For instance, starting from 0.5, the sequence progresses as follows: -0.75 -> -0.4375 -> -0.808594 -> -0.346176 -> -0.880162 ->  -0.225315 -> -0.949233 -> -0.0989567 ->  -0.990208 ->  -0.0194881 -> -0.99962 -> -0.000759856 -> -0.999999 and so on. Initially the orbit appears chaotic, but in the limit it stabilizes as a periodically stable orbit oscillating between 0 and -1.
Simple subtraction of 1 from the previous function has made orbit's behavior far more complex. Extending this analysis from the real axis to the complex plane and broadening our function f(x) to whole family of iterative functions f(z_k) = z_k-1^2 + c, where z is a complex variable and c is a complex constant, we can find infinitely inticate behavior of orbits. Plotting them as a color map will allow us to see the structure of those unimaginably complex sets.

As we observed in simpler examples, the divergence of orbits originating at points equal or beyond 1 from the origin of real axis is intuitively anticipated and this expectation is readily justified. This principle seamlessly extends to the complex plane. It is proven, that for orbits exceeding a radius of 2 it is guaranteed to diverge to infinity. It is noteworthy that if we adjust our definition of Julia sets to include trigonometric functions, exemplified by f(z_k) = sin(z_k-1)*c, the radius of divergence will need to be adjusted. 



## Basic renderer
Armed with the knowledge gained we can now proceed to implement the fractal renderer. Following libraries will be essential to utilize:

```python
import numpy as np                              # For array manipulation
from PIL import Image                           # For image processing
import matplotlib.pyplot as plt                 # For applying colormaps
from sympy import sympify, lambdify, symbols    # For symbolic mathematics
```

Additionally few other libraries are employed for final renderer. However the scope of this paper will be centered around explaining basic functionality of renderer.

A critical component to begin with is building a function computing the orbits for a given sympy function, this is crucial for determining membership in Julia set. For that we'll need to initialize several constants:

```python
max_iterations = 256
max_magnitude = 2
attractor_str = 'z^2 + const' # initial definition of atractor function
const = 0.4 + 0.4j
res_w, res_h = 1000, 1000 # resolution of outputed file in pixels
re_min, re_max, im_min, im_max = -2, 2, -2, 2 # complex plane range to be rendered
```

Subsequently we can compute our function, using sympy symbolic abilities, to be a callable object compatible with numpy's vectorized operations:

```python
# lambda like callable compatible with numpy vectorized operations
attractor = lambda x1, x2: lambdify(symbols('z const'), sympify(attractor_str), 'numpy')(x1, x2)
```

Finally we can write our function, utilizing vectorized operations:

```python
def if_in_Julia_set(z_arr:np.array, data:np.array):
    '''
    Calculates if Julia set contains a given point.
    Uses sympy expression for atractor function.
    
    Operates on passed data array !!!

    Parameters:
    - z_arr: array of complex numbers corresponding to pixels (np.array)
    - data: array to populate with iterations till exceeding max_magnitude or max_ieration if not exceeded (np.array)
    '''

    # initialize helper array
    not_exceeding = np.ones_like(data, dtype=bool)

    # iterate till exceeding max_magnitude or max_ieration if not exceeded 
    for _ in np.arange(max_iterations):

        # evaluate atractor function for relevant pixels, for current iteration
        z_arr = np.where(not_exceeding, attractor(z_arr, const), z_arr)

        # mark points exceeding max_magnitude
        not_exceeding = ~(np.abs(z_arr) > max_magnitude)

        # update data
        data[not_exceeding] += 1

    # adjust data to prevent unint8 overflow
    data[data == max_iterations] = max_iterations-1
```

Now let's introduce the rendering function:

```python
def render_vectorwise(data:np.array) -> np.array:
    '''Renders Julia set as numpy array'''

    # initialize array of complex numbers corresponding to pixels
    # np.linspace creates array of evenly spaced numbers over resoluton range
    # np.newaxis adds new axis (column vector) to array
    # data contains complex numbers corresponding to pixels
    # max/min swaped beacuse rendering goes top to bottom
    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
            * np.linspace(im_min, im_max, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data)
```

Remembering that the Julia set is the border between bounded and divergent orbits, we can plot our calculations using color maps that will show not only divergence, but also the amount of iterations it took the orbit to exceed our maximum magnitude of 2 and save it as a PNG image:

```python
def render(color_map:str="twilight_shifted") -> None:
    '''Renders Julia set into .png file'''

    # initialize image
    image = Image.new('RGB', (res_h, res_w))

    # initialize data
    data = np.zeros((res_h, res_w), dtype=np.uint8)

    # create data:
    render_vectorwise(data)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # save data to image
    image = Image.fromarray(pixels, 'RGB')
    image.save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.png")
```

And let's plot!

```python
render()
```

render_1



# Visual analisis

Now that we have developed a tool capable of rendering a visual representations of Julia sets, we can conduct visual analysis of these mathematical entities.
A key observation to start from is that varying `const` argument yields different fractal patterns. For instance:

const = 0.285 + 0.01j: render_2

const = -0.835 - 0.2321j: render_3

const = 0.35 + 0.35j: render_4

const = -0.4 + 0.6j: render_5

Furthermore it's noteworthy that in the vicinity of the Mandelbrot set's boundary we can find most visually intricate patterns of Julia sets. 
This observation stems from the relationship between the two explained earlier. The Mandelbrot set is generated by iterative function, that takes `z=0` as a starting argument and varying the `const` value. Hence, for a given `const` value the orbit of Julia set is the same as for Mandelbrot set. Consequently for values inside the Mandelbrot set Julia sets won't be as visually stimulating as near it's boundary. The impression will be similar for values distant from the Mandelbrot set's boundary:

const = 0: render_6

const = 10 + 10j: render_7


Most fascinating geometries can be discovered near the boundary of the Mandelbrot set. Utilizing the near-circular, cardioid-like shape of the Mandelbrot, we can generate captivating plots for values of `const` which magnitude equal to approximate radius of the boundary. To enhance ease of exploring thoose shapes and underscore the dynamic nature of the subject, we can animate it into a single GIF file.
For that we'll need to adjust our code by adding one more outer loop generating subsequent images to be concatenated to a single GIF file.

To accomodate the dynamic visualization for varying value of `const` parameter, we refine the existing function for determining Julia set membership:

```python
def if_in_Julia_set(z_arr:np.array, data:np.array, curr_const:complex=None):
    '''
    Calculates if Julia set contains a given point.
    Uses sympy expression for atractor function.
    
    Operates on passed data array !!!

    Parameters:
    - z_arr: array of complex numbers corresponding to pixels (np.array)
    - data: array to populate with iterations till exceeding max_magnitude or max_ieration if not exceeded (np.array)
    '''

    # adjustment for animated renders
    if not curr_const: curr_const = const

    # initialize helper array
    not_exceeding = np.ones_like(data, dtype=bool)

    # iterate till exceeding max_magnitude or max_ieration if not exceeded 
    for _ in np.arange(max_iterations):

        # evaluate atractor function for relevant pixels, for current iteration
        z_arr = np.where(not_exceeding, attractor(z_arr, curr_const), z_arr)

        # mark points exceeding max_magnitude
        not_exceeding = ~(np.abs(z_arr) > max_magnitude)

        # update data
        data[not_exceeding] += 1

    # adjust data to prevent unint8 overflow
    data[data == max_iterations] = max_iterations-1
```

We will also need to adjust the rendering function to return an `Image` object, but to ensure simplicity and readability of the existing code, we can simply design it from scratch:

```python
def render_frame(current_const:complex=0+0j, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint8)

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, current_const)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # return image
    return Image.fromarray(pixels, 'RGB')
```

Finally we can cover it all up with an outer layer that is a loop through frames that will compute consecutive values of `const` and call the above function for render image, to concatenate them all into a single GIF file:

```python
def render_gif(frames_amount:int=200, frame_duration:int=25):

    magnitude = 0.8

    # const value list
    const_values = magnitude * np.exp(1j * np.linspace(0, 2 * np.pi, frames_amount))

    frames = []
    for i in range(frames_amount):
        
        curr_const = const_values[i]

        frames.append(render_frame(curr_const))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)
```

Culminating it all up, the only thing to do is to execute the rendering of our animation!

```python
render_gif()
```

render_8.gif


# Fractals and their non-intuitive dimensionallity

One last thing I wanted to mention in this paper is the dimensionality of fractals. When it comes to these unusual mathematical entities it starts to make sense to talk about fractional dimensions (Minkowski-Bouligand dimension, Hausdorff dimension, etc,..). At first glance the notion of fractional dimensions may seem perplexing, yet it will become more coherent after short explanation. These shapes, as our examples have shown, can exhibit infinitely intricate boundary and so have an infinite length in a finite area, essentially bridging the gap between conventional dimensions. It can be achived by staying irregular at arbitrary scale. This behavior can be seen on the file below:

render_9.gif

(Note: The code generating such a zoom effect on the Mandelbrot set is pretty straightforward and will be ommited here for brevity. However I remind the reader that all renders and the codebase to create them is contained in this repository and can be easily explored)

One of the more difficult challenges with computing such is that for really deep zooms the fine line between divergent and non divergent orbits can be overwhelmed by ever so slowly diverging orbits. Currently we determine divergence by analyzing first `256` iterations of the function, but to visualize (even if only approximately) infinite complexity we'll need to adjust this parameter and to do so we might need to refine the types we're using, from uint8 (capable of storing 256 values) to uint16 (capable of storing 65_536 values).

Once again we will need to refine the existing function for determining Julia set membership by adding `curr_iter` parameter, that controls how many iterations infer divergence:

```python
def if_in_Julia_set(z_arr:np.array, data:np.array, curr_const:complex=None, curr_iter:int=256):
    '''
    Calculates if Julia set contains a given point.
    Uses sympy expression for atractor function.
    
    Operates on passed data array !!!

    Parameters:
    - z_arr: array of complex numbers corresponding to pixels (np.array)
    - data: array to populate with iterations till exceeding max_magnitude or max_ieration if not exceeded (np.array)
    '''

    # adjustment for animated renders
    if not curr_const: curr_const = const
    max_iterations = curr_iter

    # initialize helper array
    not_exceeding = np.ones_like(data, dtype=bool)

    # iterate till exceeding max_magnitude or max_ieration if not exceeded 
    for _ in np.arange(max_iterations):

        # evaluate atractor function for relevant pixels, for current iteration
        z_arr = np.where(not_exceeding, attractor(z_arr, curr_const), z_arr)

        # mark points exceeding max_magnitude
        not_exceeding = ~(np.abs(z_arr) > max_magnitude)

        # update data
        data[not_exceeding] += 1

        # break the loop if all elements exceeded given magnitude
        if not any(not_exceeding): break

    # adjust data to prevent unint8 overflow
    data[data == max_iterations] = max_iterations-1
```

Also we will need to adjust our `render_frame` function to operate on a more capacious data type unit16:

```python
def render_frame_uint16(current_iterations_max:int=256, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint16) # here we did changed types

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, const, current_iterations_max)

    # map data to colors
    # normalize orbits 
    normalized_orbits = data / current_iterations_max
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * 255).astype(np.uint8) # here we did NOT change types

    # return image
    return Image.fromarray(pixels, 'RGB')
```

Last but not least, we can rewrite `render_gif` to update value of `max_iter` instead of `const`. We will update it logarithmically, because changes at the beginning of the spectrum are more visible than far into the large values regime. Well, here it is:

```python
def render_gif(frames_amount:int=500, frame_duration:int=25, log_2_max_iter_start:int=4, log_2_max_iter_end:int=12):

    # spread vaalues of max iter logarithmically to better see first elements
    # log_2_max_iter_start  - start of logarythmic scale (2^log_...)
    # log_2_max_iter_end    - end of logarythmic scale (2^log_...)
    # frames_amount         - how many elements to generate
    # False                 - not including log_2_max_iter_end as element (ensures not getting to limit of uint16 data type)
    # int                   - casts elements too ints
    max_iter_values = np.logspace(log_2_max_iter_start, log_2_max_iter_end, frames_amount, False, 2, int)
    
    frames = []
    for i in range(frames_amount):
        
        curr_iter = max_iter_values[i]

        frames.append(render_frame_uint16(curr_iter, "twilight"))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)
```

And run!

```python
render_gif()
```

render_10.gif

# Afterword

It is our  sincere hope that this exploration of the mesmerizing world of fractals was as engaging and as captivating for the reader as for the authors of this paper. For those whose curiosity remains unquenched, we will end this paper with few more examples of those marvellous visuals and leave the reader with tools to dive deeper into the realm of fractals.

This repository contains a codebase with a comprehensive suite of tools designed for fractal generation, with different parameters to experiment with. The repository is an ongoing project and gets updates on a weekly basis, but it already contains a rich collection of functionalities. From color mapping's operations to creating visuals based on trigonometric functions and vastly more. We wholeheartedly encourage the reader to pause and dive into the creative process of fractal exploration.

Your insights, ideas, suggestions and recommendations are highly valued. If you are inspired to code with us, have recommendations to share, or wish to propose improvements, we eagerly invite you to submit a pull request or reach out to us via email at wojciech.kosnik.kowalczuk@gmail.com.

In [ ]:
# BASIC RENDERER

In [2]:
# IMPORTS
import numpy as np                              # For array manipulation
from PIL import Image                           # For image processing
import matplotlib.pyplot as plt                 # For applying colormaps
from sympy import sympify, lambdify, symbols    # For symbolic mathematics

In [73]:
# INITIALIZATION OF CONSTANTS
max_iterations = 256
max_magnitude = 2
attractor_str = 'z^2 + const' # initial definition of atractor function
const = -0.29609091 + 0.62491j 
res_w, res_h = 200, 200 # px
re_min, re_max, im_min, im_max = -0.9, 0.3, -0.3, 0.9 # complex plane range to be rendered

# lambda like callable compatible with numpy vectorized operations
attractor = lambda x1, x2: lambdify(symbols('z const'), sympify(attractor_str), 'numpy')(x1, x2)

In [57]:
# COMPUTING ORBITS
def if_in_Julia_set(z_arr:np.array, data:np.array, curr_const:complex=None, curr_iter:int=256):
    '''
    Calculates if Julia set contains a given point.
    Uses sympy expression for atractor function.
    
    Operates on passed data array !!!

    Parameters:
    - z_arr: array of complex numbers corresponding to pixels (np.array)
    - data: array to populate with iterations till exceeding max_magnitude or max_ieration if not exceeded (np.array)
    '''

    # adjustment for animated renders
    if not curr_const: curr_const = const
    max_iterations = curr_iter

    # initialize helper array
    not_exceeding = np.ones_like(data, dtype=bool)

    # iterate till exceeding max_magnitude or max_ieration if not exceeded 
    for _ in np.arange(max_iterations):

        # evaluate atractor function for relevant pixels, for current iteration
        z_arr = np.where(not_exceeding, attractor(z_arr, curr_const), z_arr)

        # mark points exceeding max_magnitude
        not_exceeding = ~(np.abs(z_arr) > max_magnitude)

        # update data
        data[not_exceeding] += 1

        # break the loop if all elements exceeded given magnitude
        if not any(not_exceeding): break

    # adjust data to prevent unint8 overflow
    data[data == max_iterations] = max_iterations-1
    

In [21]:
# RENDERING AND SAVING TO .png
def render_vectorwise(data:np.array) -> np.array:
    '''Renders Julia set as numpy array'''

    # initialize array of complex numbers corresponding to pixels
    # np.linspace creates array of evenly spaced numbers over resoluton range
    # np.newaxis adds new axis (column vector) to array
    # data contains complex numbers corresponding to pixels
    # max/min swaped beacuse rendering goes top to bottom
    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
            * np.linspace(im_max, im_min, res_h)[:, np.newaxis] 
    
    # calculate orbits
    if_in_Julia_set(z_arr, data)

def render(color_map:str="twilight_shifted") -> None:
    '''Renders Julia set into .png file'''

    # initialize image
    image = Image.new('RGB', (res_h, res_w))

    # initialize data
    data = np.zeros((res_h, res_w), dtype=np.uint8)

    # create data:
    render_vectorwise(data)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # save data to image
    image = Image.fromarray(pixels, 'RGB')
    image.save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.png")

In [22]:
# EXECUTE
render()

In [23]:
# VISUAL ANLYSIS

In [24]:
# RENDERING SINGLE FRAME FOR ANIMATION
def render_frame(current_const:complex=0+0j, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint8)

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, current_const)

    # map data to colors
    # normalize orbits
    normalized_orbits = data / max_iterations
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * max_iterations).astype(np.uint8) # remove alpha channel

    # return image
    return Image.fromarray(pixels, 'RGB')

In [25]:
# RENDERING INTO GIF FILE
def render_gif(frames_amount:int=200, frame_duration:int=25):

    magnitude = 0.8

    # const value list
    const_values = magnitude * np.exp(1j * np.linspace(0, 2 * np.pi, frames_amount))

    frames = []
    for i in range(frames_amount):
        
        curr_const = const_values[i]

        frames.append(render_frame(curr_const))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)

In [26]:
# EXECUTE
render_gif(2,2)

In [27]:
# FRACTALS UNINTUITIVE DIMENTION

In [77]:
# RENDERING SINGLE FRAME FOR ANIMATION
def render_frame_uint16(current_iterations_max:int=256, color_map:str="twilight_shifted") -> Image:
    '''Renders Julia set as numpy array'''

    data = np.zeros((res_h, res_w), dtype=np.uint16)

    z_arr = np.linspace(re_min, re_max, res_w) + 1j \
        * np.linspace(im_max, im_min, res_h)[:, np.newaxis]
    
    # calculate orbits
    if_in_Julia_set(z_arr, data, const, current_iterations_max)

    # map data to colors
    # normalize orbits 
    normalized_orbits = data / current_iterations_max
    # get colormap
    cmap = plt.colormaps[color_map]
    # map orbits
    pixels = (cmap(normalized_orbits)[:,:,:3] * 255).astype(np.uint8) # remove alpha channel

    # return image
    return Image.fromarray(pixels, 'RGB')

In [78]:
# RENDERING INTO GIF FILE
def render_gif(frames_amount:int=500, frame_duration:int=25, log_2_max_iter_start:int=4, log_2_max_iter_end:int=12):

    # spread vaalues of max iter logarithmically to better see first elements
    # log_2_max_iter_start  - start of logarythmic scale (2^log_...)
    # log_2_max_iter_end    - end of logarythmic scale (2^log_...)
    # frames_amount         - how many elements to generate
    # False                 - not including log_2_max_iter_end as element (ensures not getting to limit of uint16 data type)
    # int                   - casts elements too ints
    max_iter_values = np.logspace(log_2_max_iter_start, log_2_max_iter_end, frames_amount, False, 2, int)
    
    frames = []
    for i in range(frames_amount):
        
        curr_iter = max_iter_values[i]

        frames.append(render_frame_uint16(curr_iter, "twilight"))

    frames[0].save(f"Julia_set_{attractor_str}_res_{res_w}x{res_h}.gif", format='GIF', append_images=frames[1:], \
                    save_all=True, duration=frame_duration, loop=0)
    


In [ ]:
# EXECUTE
render_gif(10, 500)